## 바다나우 어텐션(Bahdanau Attention) 번역

###1. 데이터 로드 및 전처리

약 19만개의 병렬 문장 샘플을 포함하고 있습니다. 데이터를 읽고 전처리를 진행해보겠습니다. 앞으로의 코드에서 src는 source의 줄임말로 입력 문장을 나타내며, tar는 target의 줄임말로 번역하고자 하는 문장을 나타냅니다.

In [1]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import torch
from collections import Counter
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

약 19만개의 데이터 중 33,000개의 샘플만을 사용할 예정입니다.

In [2]:
num_samples = 33000

In [3]:
!wget -c http://www.manythings.org/anki/fra-eng.zip && unzip -o fra-eng.zip

--2025-06-26 11:19:38--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8143096 (7.8M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.77M  4.02MB/s    in 1.9s    

2025-06-26 11:19:40 (4.02 MB/s) - ‘fra-eng.zip’ saved [8143096/8143096]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


전처리 함수들을 구현합니다. 구두점 등을 제거하거나 단어와 구분해주기 위한 전처리입니다.

In [4]:
def unicode_to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


In [5]:
def preprocess_sentence(sent):
  # 악센트 삭제 함수 호출
  sent = unicode_to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백을 만듭니다.
  # Ex) "he is a boy." => "he is a boy ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent


In [6]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):
      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()]
      tar_line_out = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break

  return encoder_input, decoder_input, decoder_target


구현한 전처리 함수들을 임의의 문장을 입력으로 테스트해봅시다.

In [7]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))


전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [8]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


단어로부터 정수를 얻는 딕셔너리. 즉, 단어 집합(Vocabulary)을 만들어봅시다. 이를 위한 함수로 build_vocab()을 구현합니다. build_vocab은 입력된 데이터로부터 단어의 등장 빈도순으로 정렬 후에 등장 빈도가 높은 순서일 수록 낮은 정수를 부여합니다. 이때, 패딩 토큰을 위한 <PAD> 토큰은 0번, OOV에 대응하기 위한 <UNK> 토큰은 1번에 할당합니다. 이렇게 되면 빈도수가 가장 높은 단어는 정수가 2번, 빈도수가 두번 째로 많은 단어는 정수 3번이 할당됩니다.

In [9]:
def build_vocab(sents):
  word_list = []

  for sent in sents:
      for word in sent:
        word_list.append(word)

  # 각 단어별 등장 빈도를 계산하여 등장 빈도가 높은 순서로 정렬
  word_counts = Counter(word_list)
  vocab = sorted(word_counts, key=word_counts.get, reverse=True)

  word_to_index = {}
  word_to_index['<PAD>'] = 0
  word_to_index['<UNK>'] = 1

  # 등장 빈도가 높은 단어일수록 낮은 정수를 부여
  for index, word in enumerate(vocab) :
    word_to_index[word] = index + 2

  return word_to_index


영어를 위한 단어 집합 src_vocab과 프랑스어를 이용한 단어 집합 tar_vocab를 만들어봅시다. 구현 방식에 따라서는 하나의 단어 집합으로 만들어도 상관없으며 이는 선택의 차이입니다.

In [10]:
src_vocab = build_vocab(sents_en_in)
tar_vocab = build_vocab(sents_fra_in + sents_fra_out)

src_vocab_size = len(src_vocab)
tar_vocab_size = len(tar_vocab)
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))


영어 단어 집합의 크기 : 4498, 프랑스어 단어 집합의 크기 : 7895


정수로부터 단어를 얻는 딕셔너리를 각각 만들어줍니다. 이들은 훈련을 마치고 예측값과 실제값을 비교하는 단계에서 사용됩니다.

In [11]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

def texts_to_sequences(sents, word_to_index):
  encoded_X_data = []
  for sent in tqdm(sents):
    index_sequences = []
    for word in sent:
      try:
        index_sequences.append(word_to_index[word])
      except KeyError:
        index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data

In [12]:
encoder_input = texts_to_sequences(sents_en_in, src_vocab)
decoder_input = texts_to_sequences(sents_fra_in, tar_vocab)
decoder_target = texts_to_sequences(sents_fra_out, tar_vocab)


100%|██████████| 33000/33000 [00:00<00:00, 169676.86it/s]


In [13]:
# 상위 5개의 샘플에 대해서 정수 인코딩 전, 후 문장 출력
# 인코더 입력이므로 <sos>나 <eos>가 없음
for i, (item1, item2) in zip(range(5), zip(sents_en_in, encoder_input)):
    print(f"Index: {i}, 정수 인코딩 전: {item1}, 정수 인코딩 후: {item2}")


Index: 0, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [27, 2]
Index: 1, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [27, 2]
Index: 2, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [27, 2]
Index: 3, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [27, 2]
Index: 4, 정수 인코딩 전: ['hi', '.'], 정수 인코딩 후: [740, 2]


In [14]:
def pad_sequences(sentences, max_len=None):
  # 최대 길이 값이 주어지지 않을 경우 데이터 내 최대 길이로 패딩
  if max_len is None:
    max_len = max([len(sentence) for sentence in sentences])

  features = np.zeros((len(sentences), max_len), dtype=int)
  for index, sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index, :len(sentence)] = np.array(sentence)[:max_len]

  return features

In [15]:
encoder_input = pad_sequences(encoder_input)
decoder_input = pad_sequences(decoder_input)
decoder_target = pad_sequences(decoder_target)

In [16]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (33000, 7)
디코더의 입력의 크기(shape) : (33000, 16)
디코더의 레이블의 크기(shape) : (33000, 16)


테스트 데이터를 분리하기 전 데이터를 섞어줍니다. 이를 위해서 순서가 섞인 정수 시퀀스 리스트를 만듭니다.

In [17]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스: ', indices)

랜덤 시퀀스:  [24514 15865 25010 ... 27406   103 14338]


이를 데이터셋의 순서로 지정해주면 샘플들이 기존 순서와 다른 순서로 섞이게 됩니다.

In [18]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]


임의로 30,997번째 샘플을 출력해봅시다. 이때 decoder_input과 decoder_target은 데이터의 구조상으로 앞에 붙은 <sos> 토큰과 뒤에 붙은 <eos>을 제외하면 동일한 시퀀스를 가져야 합니다.

In [19]:
print([index_to_src[word] for word in encoder_input[30997]])
print([index_to_tar[word] for word in decoder_input[30997]])
print([index_to_tar[word] for word in decoder_target[30997]])


['tom', 'talks', 'tough', '.', '<PAD>', '<PAD>', '<PAD>']
['<sos>', 'tom', 'parle', 'comme', 'un', 'dur', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['tom', 'parle', 'comme', 'un', 'dur', '.', '<eos>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [20]:
# 33,000개의 10%에 해당되는 3,300개의 데이터를 테스트 데이터로 사용합니다
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)


검증 데이터의 개수 : 3300


In [21]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]


In [22]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)


훈련 source 데이터의 크기 : (29700, 7)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 7)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


###2. 기계 번역기 만들기
임베딩 벡터의 차원은 256, 은닉 상태의 차원 또한 256으로 지정합니다.

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

embedding_dim = 256
hidden_units = 256

임베딩 층(Embedding layer)과 LSMT 층으로 구성되며 입력 문장은 임베딩 층을 통해 각 단어가 임베딩되고, 인코더의 LSTM을 통과합니다.

In [24]:
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embedding_dim, hidden_units):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(src_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)

    def forward(self, x):
        # x.shape == (batch_size, seq_len, embedding_dim)
        x = self.embedding(x)
        # hidden.shape == (1, batch_size, hidden_units), cell.shape == (1, batch_size, hidden_units)
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden, cell


어텐션 메커니즘 자체가 디코더 단에서 시작되기 때문인데, 컨텍스트 벡터를 얻기 위한 연산 과정 자체는 바이트페어인코딩에서 설명한 순서대로 진행됩니다. 우선 디코더의 은닉 상태와 인코더의 모든 시점의 은닉 상태간의 내적(dot product)를 통해서 어텐션 스코어(attention_scores)를 구합니다.

그 후 어텐션 스코어를 소프트맥스(softmax) 함수를 통과시켜 어텐션 가중치(attention_weights)를 얻습니다. 어텐션 가중치는 어텐션 메커니즘에서 Value에 해당하는 인코더의 모든 시점의 은닉 상태와 다시 각각 곱해지고 이를 모두 더하여 컨텍스트 벡터(context_vector)를 얻습니다.

이 컨텍스트 벡터를 사용하는 방법은 어텐션 메커니즘을 구현하기 나름인데, 아래의 코드에서는 15-02에서의 설명과 같이 임베딩 벡터(아래에서는 변수 x)와 연결되어 입력으로 사용합니다.

In [25]:
class Decoder(nn.Module):
    def __init__(self, tar_vocab_size, embedding_dim, hidden_units):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(tar_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim + hidden_units, hidden_units, batch_first=True)
        self.fc = nn.Linear(hidden_units, tar_vocab_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, encoder_outputs, hidden, cell):
        x = self.embedding(x)

        # Dot product attention
        # attention_scores.shape: (batch_size, source_seq_len, 1)
        attention_scores = torch.bmm(encoder_outputs, hidden.transpose(0, 1).transpose(1, 2))
        # transpose() 함수는 차원의 순서를 바꿈
        # torch.bmm()은 batch matrix multiplication을 의미, 이름 그대로 "batch 단위로 행렬 곱을 수행"하는 함수
        # attention_weights.shape: (batch_size, source_seq_len, 1)

        attention_weights = self.softmax(attention_scores)

        # context_vector.shape: (batch_size, 1, hidden_units)
        context_vector = torch.bmm(attention_weights.transpose(1, 2), encoder_outputs)

        # Repeat context_vector to match seq_len
        # context_vector_repeated.shape: (batch_size, target_seq_len, hidden_units)
        seq_len = x.shape[1]
        context_vector_repeated = context_vector.repeat(1, seq_len, 1)
        # repeat() 함수는 텐서의 데이터를 지정한 차원 방향으로 반복해서 복제하는 함수예요. 쉽게 말해, 텐서를 복사해서 크기를 키우는 데 쓰입니다

        # Concatenate context vector and embedded input
        # x.shape: (batch_size, target_seq_len, embedding_dim + hidden_units)
        x = torch.cat((x, context_vector_repeated), dim=2)

        # output.shape: (batch_size, target_seq_len, hidden_units)
        # hidden.shape: (1, batch_size, hidden_units)
        # cell.shape: (1, batch_size, hidden_units)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))

        # output.shape: (batch_size, target_seq_len, tar_vocab_size)
        output = self.fc(output)

        return output, hidden, cell


인코더와 디코더를 연결하여 Seq2Seq 모델을 완성합니다. 결국 어텐션 메커니즘 자체가 정의되는 것은 디코더 클래스 내부의 코드입니다.

In [26]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        encoder_outputs, hidden, cell = self.encoder(src)
        output, _, _ = self.decoder(trg, encoder_outputs, hidden, cell)
        return output

encoder = Encoder(src_vocab_size, embedding_dim, hidden_units)
decoder = Decoder(tar_vocab_size, embedding_dim, hidden_units)
model = Seq2Seq(encoder, decoder)

loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters())


디코더는 인코더의 마지막 은닉 상태로부터 초기 은닉 상태를 얻습니다. 디코더도 은닉 상태, 셀 상태를 리턴하기는 하지만 훈련 과정에서는 사용하지 않습니다. seq2seq의 디코더는 기본적으로 각 시점마다 다중 클래스 분류 문제를 풀고있습니다. 매 시점마다 프랑스어 단어 집합의 크기(tar_vocab_size)의 선택지에서 단어를 1개 선택하여 이를 이번 시점에서 예측한 단어로 택합니다. 다중 클래스 분류 문제이므로 손실 함수를 크로스 엔트로피 함수를 사용합니다.

In [27]:
# 평가 함수를 작성합니다.
def evaluation(model, dataloader, loss_function, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_count = 0

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, decoder_targets in dataloader:
            encoder_inputs = encoder_inputs.to(device)
            decoder_inputs = decoder_inputs.to(device)
            decoder_targets = decoder_targets.to(device)

            # 순방향 전파
            # outputs.shape == (batch_size, seq_len, tar_vocab_size)
            outputs = model(encoder_inputs, decoder_inputs)

            # 손실 계산
            # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
            # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
            loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
            total_loss += loss.item()

            # 정확도 계산 (패딩 토큰 제외)
            mask = decoder_targets != 0
            total_correct += ((outputs.argmax(dim=-1) == decoder_targets) * mask).sum().item()
            total_count += mask.sum().item()

    return total_loss / len(dataloader), total_correct / total_count


각각의 데이터를 파이토치 텐서로 변환하고 배치 크기 128로 데이터로더로 변환합니다. 그후 학습 에포크를 30으로 설정합니다.

In [28]:
encoder_input_train_tensor = torch.tensor(encoder_input_train, dtype=torch.long)
decoder_input_train_tensor = torch.tensor(decoder_input_train, dtype=torch.long)
decoder_target_train_tensor = torch.tensor(decoder_target_train, dtype=torch.long)

encoder_input_test_tensor = torch.tensor(encoder_input_test, dtype=torch.long)
decoder_input_test_tensor = torch.tensor(decoder_input_test, dtype=torch.long)
decoder_target_test_tensor = torch.tensor(decoder_target_test, dtype=torch.long)

# 데이터셋 및 데이터로더 생성
batch_size = 128

train_dataset = TensorDataset(encoder_input_train_tensor, decoder_input_train_tensor, decoder_target_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = TensorDataset(encoder_input_test_tensor, decoder_input_test_tensor, decoder_target_test_tensor)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# 학습 설정
num_epochs = 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(4498, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(7895, 256, padding_idx=0)
    (lstm): LSTM(512, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=7895, bias=True)
    (softmax): Softmax(dim=1)
  )
)

모델을 훈련합니다. 128개의 배치 크기(128개씩 데이터를 병렬로 학습)로 총 50 에포크 학습합니다. 검증 데이터로 훈련이 제대로 되고있는지 모니터링하겠습니다.

In [29]:
# Training loop
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # 훈련 모드
    model.train()

    for encoder_inputs, decoder_inputs, decoder_targets in train_dataloader:
        encoder_inputs = encoder_inputs.to(device)
        decoder_inputs = decoder_inputs.to(device)
        decoder_targets = decoder_targets.to(device)

        # 기울기 초기화
        optimizer.zero_grad()

        # 순방향 전파
        # outputs.shape == (batch_size, seq_len, tar_vocab_size)
        outputs = model(encoder_inputs, decoder_inputs)

        # 손실 계산 및 역방향 전파
        # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
        # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
        loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
        loss.backward()

        # 가중치 업데이트
        optimizer.step()

    train_loss, train_acc = evaluation(model, train_dataloader, loss_function, device)
    valid_loss, valid_acc = evaluation(model, valid_dataloader, loss_function, device)

    print(f'Epoch: {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}')

    # 검증 손실이 최소일 때 체크포인트 저장
    if valid_loss < best_val_loss:
        print(f'Validation loss improved from {best_val_loss:.4f} to {valid_loss:.4f}. 체크포인트를 저장합니다.')
        best_val_loss = valid_loss
        torch.save(model.state_dict(), 'best_model_checkpoint.pth')


Epoch: 1/30 | Train Loss: 2.8532 | Train Acc: 0.5449 | Valid Loss: 2.9914 | Valid Acc: 0.5344
Validation loss improved from inf to 2.9914. 체크포인트를 저장합니다.
Epoch: 2/30 | Train Loss: 2.1952 | Train Acc: 0.6179 | Valid Loss: 2.4566 | Valid Acc: 0.5977
Validation loss improved from 2.9914 to 2.4566. 체크포인트를 저장합니다.
Epoch: 3/30 | Train Loss: 1.7621 | Train Acc: 0.6594 | Valid Loss: 2.1504 | Valid Acc: 0.6251
Validation loss improved from 2.4566 to 2.1504. 체크포인트를 저장합니다.
Epoch: 4/30 | Train Loss: 1.4400 | Train Acc: 0.7027 | Valid Loss: 1.9472 | Valid Acc: 0.6527
Validation loss improved from 2.1504 to 1.9472. 체크포인트를 저장합니다.
Epoch: 5/30 | Train Loss: 1.1832 | Train Acc: 0.7443 | Valid Loss: 1.8056 | Valid Acc: 0.6703
Validation loss improved from 1.9472 to 1.8056. 체크포인트를 저장합니다.
Epoch: 6/30 | Train Loss: 0.9633 | Train Acc: 0.7822 | Valid Loss: 1.6922 | Valid Acc: 0.6874
Validation loss improved from 1.8056 to 1.6922. 체크포인트를 저장합니다.
Epoch: 7/30 | Train Loss: 0.7848 | Train Acc: 0.8177 | Valid Loss: 

이는 15-02에서 확인한 기록과 비교하면 검증 데이터의 손실이 좀 더 작으며 검증 데이터의 정확도는 더 큰 것을 알 수 있습니다. 검증 데이터 손실이 가장 최소일 때의 모델을 로드하고 다시 재평가해봅시다.

In [30]:
# 모델 로드
model.load_state_dict(torch.load('best_model_checkpoint.pth'))

# 모델을 device에 올립니다.
model.to(device)

# 검증 데이터에 대한 정확도와 손실 계산
val_loss, val_accuracy = evaluation(model, valid_dataloader, loss_function, device)

print(f'Best model validation loss: {val_loss:.4f}')
print(f'Best model validation accuracy: {val_accuracy:.4f}')


Best model validation loss: 1.4769
Best model validation accuracy: 0.7249


##3. seq2seq 기계 번역기 동작하기
seq2seq는 훈련 과정(교사 강요)과 테스트 과정에서의 동작 방식이 다릅니다. 그래서 테스트 과정을 위해 모델을 다시 설계해주어야 합니다. 특히 디코더를 수정해야 합니다. 이번에는 번역 단계를 위해 모델을 수정하고 동작시켜보겠습니다.

전체적인 번역 단계를 정리하면 아래와 같습니다.

- 1 번역하고자 하는 입력 문장이 인코더로 입력되어 인코더의 마지막 시점의 은닉 상태와 셀 상태를 얻습니다.
- 2 인코더의 은닉 상태와 셀 상태, 그리고 토큰 sos를 디코더로 보냅니다.
- 3 디코더가 토큰 eos가 나올 때까지 다음 단어를 예측하는 행동을 반복합니다.

결과 확인을 위한 함수를 만듭니다. seq_to_src 함수는 영어 문장에 해당하는 정수 시퀀스를 입력받으면 정수로부터 영어 단어를 리턴하는 index_to_src를 통해 영어 문장으로 변환합니다. seq_to_tar은 프랑스어에 해당하는 정수 시퀀스를 입력받으면 정수로부터 프랑스어 단어를 리턴하는 index_to_tar을 통해 프랑스어 문장으로 변환합니다.

In [31]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_vocab['<sos>'] and encoded_word != tar_vocab['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence


25번 샘플의 정수 인코딩이 진행된 인코더 입력, 디코더 입력, 그리고 디코더의 레이블을 출력해봅시다.

In [32]:
print(encoder_input_test[25])
print(decoder_input_test[25])
print(decoder_target_test[25])

[   3  710   10 3841    2    0    0]
[   3    5   74 1261    9   25 6577    2    0    0    0    0    0    0
    0    0]
[   5   74 1261    9   25 6577    2    4    0    0    0    0    0    0
    0    0]


decode_sequence() 함수를 봅시다. 테스트 단계에서는 디코더를 매 시점 별로 컨트롤 하게 됩니다. 각 시점을 for문을 통해서 컨트롤하게 되며, 현재 시점의 예측은 다음 시점의 입력으로 사용됩니다. 여기서 사용될 변수는 decoder_input입니다.

In [33]:
def decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, max_output_len, int_to_src_token, int_to_tar_token):
  encoder_inputs = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)

  # 인코더의 초기 상태 설정
  encoder_outputs, hidden, cell = model.encoder(encoder_inputs)

  # 시작 토큰 <sos>을 디코더의 첫 입력으로 설정
  decoder_input = torch.tensor([3], dtype=torch.long).unsqueeze(0).to(device)

  decoded_tokens = []

  # for문을 도는 것 == 디코더의 각 시점
  for _ in range(max_output_len):
    output, hidden, cell = model.decoder(decoder_input, encoder_outputs, hidden, cell)

    # 소프트맥스 회귀를 수행. 예측 단어의 인덱스
    output_token = output.argmax(dim=-1).item()

    # 종료 토큰 <eos>
    if output_token == 4:
      break

    # 각 시점의 단어(정수)는 decoded_tokens에 누적하였다가 최종 번역 시퀀스로 리턴합니다.
    decoded_tokens.append(output_token)

    # 현 시점의 예측. 다음 시점의 입력으로 사용된다.
    decoder_input = torch.tensor([output_token], dtype=torch.long).unsqueeze(0).to(device)

  return ' '.join(int_to_tar_token[token] for token in decoded_tokens)



훈련 데이터에 대해서 임의로 선택한 인덱스의 샘플의 결과를 출력해봅시다.

In [34]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index]
  translated_text = decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, 20, index_to_src, index_to_tar)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",translated_text)
  print("-"*50)


입력문장 : tom has seniority . 
정답문장 : tom a de l experience . 
번역문장 : tom a de l experience .
--------------------------------------------------
입력문장 : you re all fired . 
정답문장 : vous etes tous licencies . 
번역문장 : vous etes tous licencies .
--------------------------------------------------
입력문장 : his word is law . 
정답문장 : sa parole est loi . 
번역문장 : sa parole est la loi .
--------------------------------------------------
입력문장 : you re dangerous . 
정답문장 : vous etes dangereux . 
번역문장 : vous etes dangereuses .
--------------------------------------------------
입력문장 : we aren t related . 
정답문장 : nous ne sommes pas apparentes . 
번역문장 : nous ne sommes pas apparentes .
--------------------------------------------------


테스트 데이터에 대해서 임의로 선택한 인덱스의 샘플의 결과를 출력해봅시다.

In [35]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index]
  translated_text = decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, 20, index_to_src, index_to_tar)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",translated_text)
  print("-"*50)


입력문장 : be creative . 
정답문장 : soyez creative ! 
번역문장 : soyez creatives !
--------------------------------------------------
입력문장 : tom has a big tv . 
정답문장 : tom a une grande television . 
번역문장 : tom a une grosse grosse .
--------------------------------------------------
입력문장 : no one will know . 
정답문장 : personne ne saura . 
번역문장 : personne ne le saura .
--------------------------------------------------
입력문장 : we re kidding . 
정답문장 : nous plaisantons . 
번역문장 : nous plaisantons .
--------------------------------------------------
입력문장 : take mine . 
정답문장 : prenez la mienne . 
번역문장 : prends les miennes .
--------------------------------------------------
